# Capstone Project - The Battle of Neighborhoods

### Introduction/Business Problem <br>
The scenario that we focus on in this notebook is the following:

A person/company is planning on opening a new restaurant in the city of Cleveland, Ohio, USA. <br>
Besides other important aspects, the location of the new restaurant has a huge impact on how successfull the restaurant will be. <br>
Although there might be many other factors that contribute to how well a specific location might be, the owners want to make a first choice of the location based on these two aspects: <br>
<br>
1) population: How many people live around the location? <br>
2) competition: How many other restaurants are nearby this location? <br>
<br>
In order to distinguish between different "locations", the owners decide to compare different ZIP Codes in Cleveland based on the two named aspects. <br>

### Data Section <br>
The following sources of data will be used to solve the given problem, i.e. to answer the two question above:

In the first step, we need to get the ZIP Codes for different neighborhoods in Cleveland. We get them by using the Python Librares "urllib" and "Beautiful Soap" to parse the XML-file of the website https://zipcode.org/city/OH/CLEVELAND.<br>
The page contains a list of ZIP Codes in Cleveland such as "44114", "44124" and "44144", which we will (using urllib, Beautiful Soup and Pandas) save in a DataFrame. <br>
<br>
In the next step, we will again use urllib, BeautifulSoup and Pandas to scrape the data from https://gist.github.com/erichurst/7882666. This page contains a list of ZIP Codes all over the US and their geographical coordinates, example: 44114 (zip code), 41.520126 (latitude), -81.678083 (longitude). Having transferred these information into a Pandas DataFrame, we merge them to the existing DataFrame of Cleveland's ZIP Codes.<br>
<br>
In the last step, we use the Foursquare API in order to receive venues that are nearby for each ZIP Code. We will filter the results to only include the restaurants nearby.

### Data Acquisition and Cleaning

Before we start with the "real" coding, we import necessary libraries

In [1]:
#import necessary libraries
import numpy as np              #arrays, vectors,...
import pandas as pd             #DataFrames,...
import requests                 #make url-requests
import json                     #handle json-files
import urllib.request as req    #open/read xml-files of given url
from bs4 import BeautifulSoup   #library to work with xml-files

Next, we'll start the Data Acquisition. We use BeautifulSoup and Pandas to get the ZIP Codes of Cleveland and save them in a DataFrame.

In [2]:
#define the url which contains the zip code information
url = "https://zipcode.org/city/OH/CLEVELAND"

In [3]:
#save the xml-file of the url in a variable
zip_codes = req.urlopen(url)

In [4]:
#save the xml-file as a BeautifulSoup object, so that we can work with it better in the next steps
zip_codes = BeautifulSoup(zip_codes, "lxml")

For the sake of shortness, we do not print out the whole xml-file here. <br>
The most important information is that the ZIP Codes can be extracted from lines that have the following structure: <br>
##### \<a class="List_Link_Text" href="/44101">44101 Zip Code\</a>\<br class="list"/> <br>
<br>
Therefore, we first extract those lines from the xml.

In [5]:
zip_codes_1 = zip_codes.find_all("a", class_ = "List_Link_Text")

In [6]:
#print out the first five elements (rows) to analyze their structure
zip_codes_1[0:5]

[<a class="List_Link_Text" href="/areacode/216">216 Area Code</a>,
 <a class="List_Link_Text" href="/areacode/330">330 Area Code</a>,
 <a class="List_Link_Text" href="/areacode/440">440 Area Code</a>,
 <a class="List_Link_Text" href="/44101">44101 Zip Code</a>,
 <a class="List_Link_Text" href="/44102">44102 Zip Code</a>]

We see that we need to perform some split-operations on each line in order to extract the ZIP Codes. <br>
As the split-operator works on str-objects only, we transform the elements of zip_codes_1 to type string.

In [7]:
rows = []                #empty list
for x in zip_codes_1:    #for every element in zip_codes_1
    rows.append(str(x))  #transform the element into type string and save it into the list

Now we can use the split-operator on each element. <br>
We will do this on an example and then apply the operation on every element. <br>
We will use the fourth element as the example as it is the first that contains a ZIP Code.

In [8]:
#let us take a look at the fourth element
rows[3]

'<a class="List_Link_Text" href="/44101">44101 Zip Code</a>'

In [9]:
#split this string after each "/" and select the second element
rows[3].split('/')[1]

'44101">44101 Zip Code<'

In [10]:
#split the resulting string after each ' ">" ' and select the first element
rows[3].split('/')[1].split('">')[0]

'44101'

Now that we know how to extract a single ZIP Code from a single row, we apply these operations on each row and save the results in a list:

In [11]:
zip_codes_cleveland = []  #empty list, in which we will save the results of the split operations
for row in rows:
    zip_codes_cleveland.append(row.split('/')[1].split('">')[0])

#look at the resulting list
zip_codes_cleveland

['areacode',
 'areacode',
 'areacode',
 '44101',
 '44102',
 '44103',
 '44104',
 '44105',
 '44106',
 '44109',
 '44110',
 '44113',
 '44108',
 '44111',
 '44112',
 '44114',
 '44115',
 '44118',
 '44121',
 '44119',
 '44120',
 '44124',
 '44125',
 '44126',
 '44128',
 '44127',
 '44129',
 '44130',
 '44143',
 '44134',
 '44135',
 '44144',
 '44181',
 '44188',
 '44190',
 '44194',
 '44195',
 '44191',
 '44192',
 '44193',
 '44197',
 '44198',
 '44199',
 'city',
 'city',
 'city',
 'city',
 'city',
 'city',
 'city',
 'city',
 'city',
 'city',
 'city',
 'city',
 'city',
 'city',
 'city',
 'city',
 'city',
 'city']

In the output above, we see that there are some elements that contain the words 'areacode' and 'city'. We do not need/want those elements in our list. <br>
Nevertheless, we transform the whole lost into a Pandas DataFrame and we will get rid of them in the next steps.

In [12]:
#transform the list into a Pandas DataFrame
zip_cleveland = pd.DataFrame(zip_codes_cleveland)
zip_cleveland.head()  #look at first elements of Dataframe

,0
0,areacode
1,areacode
2,areacode
3,44101
4,44102


In [13]:
#rename the column
zip_cleveland.columns = ['Zip Code']

In order to get rid of the unnecessary rows ('areacode', 'city') we first create a boolean list that contains the value "True" in element i if and only if row i of the Dataframe contains the string '44'.

In [14]:
msk = [True for item in zip_cleveland['Zip Code']]

for i in range(len(zip_cleveland['Zip Code'])):
    msk[i] = '44' in zip_cleveland['Zip Code'][i]

Now we use this list to keep all rows that contain a ZIP Code and drop all others.

In [15]:
zip_cleveland = zip_cleveland[msk].reset_index(drop = True)

In [16]:
#let us take a look at the resulting DataFrame
zip_cleveland.head()

,Zip Code
0,44101
1,44102
2,44103
3,44104
4,44105


Now that we have gathered the ZIP Codes of Cleveland and put them into a Dataframe, our next goal is to add the geographical coordinates (latitude, longitude) to every ZIP Code. <br>
We will do that by scraping the website https://gist.github.com/erichurst/7882666 which contains a list/table of all ZIP Codes in the USA and their corresponding location.

In [17]:
url_zip = "https://gist.github.com/erichurst/7882666" #url that contains location data to ZIP Codes

result = req.urlopen(url_zip)                         #open url
result = BeautifulSoup(result, "lxml")                #save xml-file as BeautifulSoup object

For the sake of shortness, we again do not show the whole xml-file of the given URL. <br>
The information we need to get is saved in lines that have the following structure: <br>
##### \<td class="blob-code blob-code-inner js-file-line" id="file-us-zip-codes-from-2013-government-data-LC5849">18656,41.332524, -76.185452\</td> <br>
Therefore, we first extract those lines from the xml-file

In [18]:
result = result.find_all('td', class_ = "blob-code blob-code-inner js-file-line")

In [19]:
#let us take a look at the resulting list
result[0:5]

[<td class="blob-code blob-code-inner js-file-line" id="file-us-zip-codes-from-2013-government-data-LC1">ZIP,LAT,LNG</td>,
 <td class="blob-code blob-code-inner js-file-line" id="file-us-zip-codes-from-2013-government-data-LC2">00601,18.180555, -66.749961</td>,
 <td class="blob-code blob-code-inner js-file-line" id="file-us-zip-codes-from-2013-government-data-LC3">00602,18.361945, -67.175597</td>,
 <td class="blob-code blob-code-inner js-file-line" id="file-us-zip-codes-from-2013-government-data-LC4">00603,18.455183, -67.119887</td>,
 <td class="blob-code blob-code-inner js-file-line" id="file-us-zip-codes-from-2013-government-data-LC5">00606,18.158345, -66.932911</td>]

Again, we need to tranform the xml-lines into objects of type string, before we can use the split-operator to extract the necessary data only.

In [20]:
rows = []                 #empty list
for x in result:          #for every row in xml-file
    rows.append(str(x))   #transform row to string and save it in list

As we did when extracting the ZIP Codes from Cleveland, we again show on an example how to extract this wanted data from each element of the list.

In [21]:
#let us take a look at a single element
rows[1]

'<td class="blob-code blob-code-inner js-file-line" id="file-us-zip-codes-from-2013-government-data-LC2">00601,18.180555, -66.749961</td>'

In [22]:
#split the string after each '>' and select the second but last element
rows[1].split('>')[-2]

'00601,18.180555, -66.749961</td'

In [23]:
#split the resulting string after each ',' to separate zip code, latitude and longitude
result_1 = rows[1].split('>')[-2].split(',')
result_1 #look at resulting list

['00601', '18.180555', ' -66.749961</td']

In [24]:
list_ = []              #create empty list
list_.append(result_1)  #add the list that resulted from the split as an element to the new list

In [25]:
#transform the list into a Dataframe
pd.DataFrame(list_)

,0,1,2
0,00601,18.180555,-66.749961</td


This is the wanted format: A Dataframe that contains the ZIP Code, its latitude and its longitude in three distinct columns. <br>
We now apply that procedure to every row and store all the results in one Dataframe.

In [26]:
list_ = []                                       #create an empty list
for i in range(len(rows)):                       #loop over all "rows"
    infos_ = rows[i].split('>')[-2].split(',')   #split the row to extract zip code, latitude and longitude
    list_.append(infos_)                         #add the new infos to the list
    
df = pd.DataFrame(list_)                         #transform the list into a Dataframe

Let us use the head-function to show the result of the loop above:

In [27]:
df.head()

,0,1,2
0,ZIP,LAT,LNG</td
1,00601,18.180555,-66.749961</td
2,00602,18.361945,-67.175597</td
3,00603,18.455183,-67.119887</td
4,00606,18.158345,-66.932911</td


We see that there are still three things left to do in order to clean up the Dataframe. <br>
The first row shows the column headers. Therefore, we'll leave this row out. <br>
Secondly, we will rename the columns to ZIP, Latitude and Longitude. <br>
Additionally, the longitude items have an '</td'-ending we need to get rid of. We will do so by again using the split-operator.

In [28]:
df = df[1:]                                                    #leave out the first row
df = df.reset_index(drop = True)                               #reset index after row 0 is left out
df.columns = ['ZIP', 'Latitude', 'Longitude']                  #rename the columns

for i in range(len(df['Longitude'])):                          #for every Longitude itemget rid of the '</td'-ending,
    df['Longitude'][i] = df['Longitude'][i].split('<')[0]      #get rid of the '</td'-ending

df.head()                                                      #show the resulting datafram (first five rows)

,ZIP,Latitude,Longitude
0,00601,18.180555,-66.749961
1,00602,18.361945,-67.175597
2,00603,18.455183,-67.119887
3,00606,18.158345,-66.932911
4,00610,18.295366,-67.125135


Now we can merge the geocoordinates to the ZIP Codes of Cleveland, Ohio. <br>
We will therefore perform a left join on the Dataframe zip_cleveland on the columns Zip Code and ZIP.

In [29]:
cleveland_zip = zip_cleveland.join(df.set_index('ZIP'), on = 'Zip Code')

Let us look at the resulting DataFrame.

In [30]:
print('The new Dataframe consists of {} rows and {} columns'.format(cleveland_zip.shape[0],cleveland_zip.shape[1]))
cleveland_zip

The new Dataframe consists of 40 rows and 3 columns


,Zip Code,Latitude,Longitude
0,44101,41.489355,-81.667393
1,44102,41.479174,-81.740603
2,44103,41.519415,-81.642123
3,44104,41.482230,-81.626784
4,44105,41.449476,-81.630289
5,44106,41.505341,-81.605432
6,44109,41.447671,-81.694403
7,44110,41.569382,-81.564687
8,44113,41.483241,-81.697166
9,44108,41.544784,-81.607394


We see that there are still some ZIP Codes that were not assigned geo coordinates as they were not listed on https://gist.github.com/erichurst/7882666. <br>
We will leave these ZIP Codes out of our following analysis.

In [31]:
#remove last rows from the dataframe (as they do not contain usable information)
cleveland_zip = cleveland_zip.iloc[0:29, :]

In [32]:
cleveland_zip

,Zip Code,Latitude,Longitude
0,44101,41.489355,-81.667393
1,44102,41.479174,-81.740603
2,44103,41.519415,-81.642123
3,44104,41.482230,-81.626784
4,44105,41.449476,-81.630289
5,44106,41.505341,-81.605432
6,44109,41.447671,-81.694403
7,44110,41.569382,-81.564687
8,44113,41.483241,-81.697166
9,44108,41.544784,-81.607394


For later analysis, we want to add the population of each ZIP Code to the Dataframe above. <br>
We will do so by scraping https://www.zipdatamaps.com/zipcodes-cleveland-oh.

In [33]:
url_population = "https://www.zipdatamaps.com/zipcodes-cleveland-oh"

result_pop = req.urlopen(url_population)                             #open url
result_pop = BeautifulSoup(result_pop, "lxml")                       #store the url's xml-file in a BeautifulSoup object

For better readability, we leave out the xml-file in this place. <br>
The important information is that the necessary data (population for ZIP Codes in Cleveland) is stored in a "table" of class "table table-striped table-bordered table-hover table-condensed". <br>
Therefore, we will extract this table from the xml-file:

In [34]:
result_pop = result_pop.find("table", class_= "table table-striped table-bordered table-hover table-condensed")

Inside of this table, the different rows start with "tr" and end with "/tr". <br>
So, we extract the single rows and store them in list:

In [35]:
result_pop_lines = result_pop.find_all('tr')

Let us take a look at the first five elements of the resulting list (rows of the table):

In [36]:
result_pop_lines[0:5]

[<tr><th colspan="4"><h2>List of Zipcodes in Cleveland, Ohio</h2></th></tr>,
 <tr><td>ZIP Code</td><td>ZIP Code Name</td><td>Population</td><td>Type</td></tr>,
 <tr><td><table><tr><td></td><td><a class="mapzoom" href="javascript:pz(-81.73944855,41.47610092);" id="[-81.73944855,41.47610092,14]" title="Zoom to 44102"><i class="fa fa-search-plus"></i></a></td><td><div id="zc44102"></div></td><td><a href="44102">44102</a></td></tr></table></td><td>Cleveland</td><td>45014</td><td>Non-Unique</td></tr>,
 <tr><td></td><td><a class="mapzoom" href="javascript:pz(-81.73944855,41.47610092);" id="[-81.73944855,41.47610092,14]" title="Zoom to 44102"><i class="fa fa-search-plus"></i></a></td><td><div id="zc44102"></div></td><td><a href="44102">44102</a></td></tr>,
 <tr><td><table><tr><td></td><td><a class="mapzoom" href="javascript:pz(-81.64344788,41.52030182);" id="[-81.64344788,41.52030182,14]" title="Zoom to 44103"><i class="fa fa-search-plus"></i></a></td><td><div id="zc44103"></div></td><td><a h

Our goal is to extract the ZIP Code and the population out of the relevant items of the list. <br>
To gain a better understanding, we will first execute this extraction on a single ZIP Code.

In [37]:
#extract a (relevant) element of the list (row of the table)
test_line = result_pop_lines[2]
test_line

<tr><td><table><tr><td></td><td><a class="mapzoom" href="javascript:pz(-81.73944855,41.47610092);" id="[-81.73944855,41.47610092,14]" title="Zoom to 44102"><i class="fa fa-search-plus"></i></a></td><td><div id="zc44102"></div></td><td><a href="44102">44102</a></td></tr></table></td><td>Cleveland</td><td>45014</td><td>Non-Unique</td></tr>

As we can see from the output, the different items inside a row are separated by "td". <br>
So let's split the row into the different items and store them in a list.

In [38]:
test_line_items = test_line.find_all('td')
test_line_items

[<td><table><tr><td></td><td><a class="mapzoom" href="javascript:pz(-81.73944855,41.47610092);" id="[-81.73944855,41.47610092,14]" title="Zoom to 44102"><i class="fa fa-search-plus"></i></a></td><td><div id="zc44102"></div></td><td><a href="44102">44102</a></td></tr></table></td>,
 <td></td>,
 <td><a class="mapzoom" href="javascript:pz(-81.73944855,41.47610092);" id="[-81.73944855,41.47610092,14]" title="Zoom to 44102"><i class="fa fa-search-plus"></i></a></td>,
 <td><div id="zc44102"></div></td>,
 <td><a href="44102">44102</a></td>,
 <td>Cleveland</td>,
 <td>45014</td>,
 <td>Non-Unique</td>]

We see that there are multiple items that contain the ZIP Code. We choose to extract the ZIP from the fourth item.

In [39]:
test_line_items[3]

<td><div id="zc44102"></div></td>

The ZIP Code is located between the 'zc' and the '">'. So, we get the ZIP Code by transforming the item to type string and performing the following split-operation.

In [40]:
str(test_line_items[3]).split('c')[1].split('">')[0]

'44102'

Next, we want to extract the population number. If we look at our test_line_items, we see that this information is contained in the seventh item.

In [41]:
test_line_items[6]

<td>45014</td>

We will extract the population number by applying the following split-operations.

In [42]:
str(test_line_items[6]).split('>')[1].split('<')[0]

'45014'

Now, we will execute these steps on every row of the xml-table and store the results in a list. <br>
As not all the rows conatin relevant data, we also fill a list called error_lines that will contain lines that did not deliver ZIP Codes and population.

In [43]:
df_rows =  []    #empty list, will later contain the ZIP Codes and their population
error_lines = [] #empty list, will later contain the rows that did not give information about the population of a ZIP

for test_line in result_pop_lines:                                                  #loop over all rows
    test_line_items = test_line.find_all('td')                                      #get single items of the current row
    try:
        zip_code_tmp = str(test_line_items[3]).split('c')[1].split('">')[0]         #extract ZIP Code
        pop_tmp = str(test_line_items[6]).split('>')[1].split('<')[0]               #extract population
        df_rows.append((zip_code_tmp, pop_tmp))                                     #store ZIP Code and population as a tuple in the list
    except:
        error_lines.append(test_line_items)                                         #if ZIP or population could not be extracted, save row as error line
        
df_rows                                                                             #print out the list

[('44102', '45014'),
 ('44103', '18123'),
 ('44104', '22640'),
 ('44105', '40089'),
 ('44106', '26896'),
 ('44107', '52244'),
 ('44108', '25679'),
 ('44109', '40646'),
 ('44110', '20136'),
 ('44111', '39778'),
 ('44112', '23073'),
 ('44113', '19213'),
 ('44114', '5225'),
 ('44115', '8307'),
 ('44117', '10224'),
 ('44119', '12482'),
 ('44120', '38196'),
 ('44121', '33220'),
 ('44122', '34057'),
 ('44126', '16771'),
 ('44127', '5586'),
 ('44128', '29274'),
 ('44129', '29090'),
 ('44134', '38800'),
 ('44135', '26861'),
 ('44142', '19201'),
 ('44144', '21089')]

In the next steps, the list is transformed into a Dataframe. Also we will rename the columns and take a look at the result.

In [44]:
population_df = pd.DataFrame(df_rows)

In [45]:
population_df.columns = ['Zip Code', 'Population']

In [46]:
population_df.head()

,Zip Code,Population
0,44102,45014
1,44103,18123
2,44104,22640
3,44105,40089
4,44106,26896


The extraction of the ZIP Codes' population is completed. <br>
Now, we will join the population into the existing Dataframe that already contains the Cleveland ZIP Codes along with their latitude and longitude:

In [47]:
#join Population information into the existing Dataframe
cleveland_data = cleveland_zip.join(population_df.set_index('Zip Code'), on = 'Zip Code')
#show result
cleveland_data

,Zip Code,Latitude,Longitude,Population
0,44101,41.489355,-81.667393,NaN
1,44102,41.479174,-81.740603,45014
2,44103,41.519415,-81.642123,18123
3,44104,41.482230,-81.626784,22640
4,44105,41.449476,-81.630289,40089
5,44106,41.505341,-81.605432,26896
6,44109,41.447671,-81.694403,40646
7,44110,41.569382,-81.564687,20136
8,44113,41.483241,-81.697166,19213
9,44108,41.544784,-81.607394,25679


Some ZIP Codes were not assigned a population number, as they were not listed on https://www.zipdatamaps.com/zipcodes-cleveland-oh. <br>
We will leave these ZIP Codes out of further analysis.

In [48]:
msk_pop = []

for item in cleveland_data['Population']:
    msk_pop.append(type(item) == str)
msk_pop

[False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True]

In [49]:
cleveland_data = cleveland_data[msk_pop]
cleveland_data = cleveland_data.reset_index(drop = True)
cleveland_data

,Zip Code,Latitude,Longitude,Population
0,44102,41.479174,-81.740603,45014
1,44103,41.519415,-81.642123,18123
2,44104,41.482230,-81.626784,22640
3,44105,41.449476,-81.630289,40089
4,44106,41.505341,-81.605432,26896
5,44109,41.447671,-81.694403,40646
6,44110,41.569382,-81.564687,20136
7,44113,41.483241,-81.697166,19213
8,44108,41.544784,-81.607394,25679
9,44111,41.458255,-81.788589,39778


To make our further analysis easier, we make sure that the different columns have entries of the suitable type.

In [50]:
type_dict = {'Zip Code': int,
            'Latitude': float,
            'Longitude': float,
            'Population': int}

cleveland_data = cleveland_data.astype(type_dict)

As we want to use the new information about the population in each ZIP Code for our analysis, we normalize this column using the min/max-method.

In [51]:
max_pop = max(cleveland_data['Population'])
min_pop = min(cleveland_data['Population'])

In [52]:
print('Maximum population of single zip code: ' + str(max_pop))
print('Minimum population of single zip code: ' + str(min_pop))

Maximum population of single zip code: 45014
Minimum population of single zip code: 5225


In [53]:
cleveland_data['Population_normalized'] = (cleveland_data['Population'] - min_pop) / (max_pop - min_pop)

What we have up to this point is the following Dataframe:

In [54]:
cleveland_data.head()

,Zip Code,Latitude,Longitude,Population,Population_normalized
0,44102,41.479174,-81.740603,45014,1.000000
1,44103,41.519415,-81.642123,18123,0.324160
2,44104,41.482230,-81.626784,22640,0.437684
3,44105,41.449476,-81.630289,40089,0.876222
4,44106,41.505341,-81.605432,26896,0.544648


The last information we want to add to this Dataframe is the amount of restaurants that are located in the ZIP Codes or nearby. <br>
We will do this by using the explore-endpoint of the Foursquare API (https://api.foursquare.com). We will gather all venues to each ZIP Code and then filter out those venues that do not belong the Category "Restaurant".

In [55]:
#Foursquare Credentials
CLIENT_ID = '2ZEUWWXANBTVKSR5ZZ1VA0B44PMXJEZYNDLZ1KSNRMJ4Q3P3'
CLIENT_SECRET = 'TFWU2LVU042GXAPJ1IEOHD22Y1AON1PUVRAORD1AJITQQTD0'
VERSION = '20201116'
LIMIT = 100

The following function will be used to get the venues nearby each ZIP Code and to put the information in a Dataframe

In [56]:
def get_nearby_venues(zips, latitudes, longitudes, radius = 2000):
    venues_list = []
    for zip_, lat, lon in zip(zips, latitudes, longitudes):
        url_fs = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,
                                                                                                                                     CLIENT_SECRET,
                                                                                                                                     VERSION,
                                                                                                                                     lat,
                                                                                                                                     lon,
                                                                                                                                     radius,
                                                                                                                                     LIMIT)
        results_fs = requests.get(url_fs).json()["response"]["groups"][0]["items"]
        
        venues_list.append([(zip_,
                             lat,
                             lon,
                             v["venue"]["name"],
                             v["venue"]["location"]["lat"],
                             v["venue"]["location"]["lng"],
                             v["venue"]["categories"][0]["name"]) for v in results_fs])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip Code',
                             'Latitude',
                             'Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

Now, we call the function above to get a Dataframe that includes all venues nearby each ZIP Code.

In [57]:
cleveland_venues = get_nearby_venues(cleveland_data['Zip Code'], cleveland_data['Latitude'], cleveland_data['Longitude'])

Let us look at the resulting Dataframe:

In [58]:
cleveland_venues.head()

,Zip Code,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,44102,41.479174,-81.740603,78th Street Studios,41.484175,-81.739577,Art Gallery
1,44102,41.479174,-81.740603,Local West,41.482718,-81.735676,Sandwich Place
2,44102,41.479174,-81.740603,Banter Beer and Wine,41.482838,-81.735492,Food & Drink Shop
3,44102,41.479174,-81.740603,Don's Lighthouse,41.484667,-81.746104,Seafood Restaurant
4,44102,41.479174,-81.740603,Sweet Moses,41.483694,-81.731868,Dessert Shop


We now add a column to the Dataframe above that will contain the value "True" exactly in those rows that have the word "Restaurant" in their "Venue Category". <br>
Otherwise the value will be set to "False".

In [59]:
list_cat = []

for i, cat in enumerate(cleveland_venues['Venue Category']):
    list_cat.append( ('Restaurant' in cleveland_venues['Venue Category'][i]) )

cleveland_venues['Restaurant'] = list_cat

Let us look at the result:

In [60]:
cleveland_venues.head()

,Zip Code,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Restaurant
0,44102,41.479174,-81.740603,78th Street Studios,41.484175,-81.739577,Art Gallery,False
1,44102,41.479174,-81.740603,Local West,41.482718,-81.735676,Sandwich Place,False
2,44102,41.479174,-81.740603,Banter Beer and Wine,41.482838,-81.735492,Food & Drink Shop,False
3,44102,41.479174,-81.740603,Don's Lighthouse,41.484667,-81.746104,Seafood Restaurant,True
4,44102,41.479174,-81.740603,Sweet Moses,41.483694,-81.731868,Dessert Shop,False


In order to extract those venues that belong to the Category "Restaurant" , we simply filter the rows that have the value "True" in the column "Restaurant" and store them in a new Dataframe.

In [61]:
cleveland_restaurants = cleveland_venues[cleveland_venues['Restaurant']==True]
cleveland_restaurants = cleveland_restaurants.reset_index(drop = True)
cleveland_restaurants.head()

,Zip Code,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Restaurant
0,44102,41.479174,-81.740603,Don's Lighthouse,41.484667,-81.746104,Seafood Restaurant,True
1,44102,41.479174,-81.740603,Luxe Kitchen & Lounge,41.483808,-81.731052,American Restaurant,True
2,44102,41.479174,-81.740603,Frank's Falafel House,41.481796,-81.729973,Falafel Restaurant,True
3,44102,41.479174,-81.740603,Villa Y Zapata,41.477276,-81.743501,Mexican Restaurant,True
4,44102,41.479174,-81.740603,Blue Habanero,41.484178,-81.729876,Mexican Restaurant,True


Now, let us count the amount of restaurants per each ZIP Code by grouping the above dataframe by column "ZIP Code":

In [62]:
cleveland_restaurants_count = cleveland_restaurants.groupby('Zip Code').count()
cleveland_restaurants_count = cleveland_restaurants_count[['Restaurant']]
cleveland_restaurants_count.head()

,Restaurant
Zip Code,
44102,12
44103,9
44104,3
44105,6
44106,23


The new column that contains the number of restaurants for each ZIP Code is now joint into the Main DataFrame (cleveland_data):

In [63]:
cleveland_data = cleveland_data.join(cleveland_restaurants_count, on = 'Zip Code')

In [64]:
#look at the current dataframe
cleveland_data

,Zip Code,Latitude,Longitude,Population,Population_normalized,Restaurant
0,44102,41.479174,-81.740603,45014,1.000000,12.0
1,44103,41.519415,-81.642123,18123,0.324160,9.0
2,44104,41.482230,-81.626784,22640,0.437684,3.0
3,44105,41.449476,-81.630289,40089,0.876222,6.0
4,44106,41.505341,-81.605432,26896,0.544648,23.0
5,44109,41.447671,-81.694403,40646,0.890221,8.0
6,44110,41.569382,-81.564687,20136,0.374752,4.0
7,44113,41.483241,-81.697166,19213,0.351554,22.0
8,44108,41.544784,-81.607394,25679,0.514062,3.0
9,44111,41.458255,-81.788589,39778,0.868406,14.0


We see that there is one ZIP Code (44134) that has no value in the column "Restaurant". <br>
We will set the value for this ZIP Code to zero:

In [65]:
for i in range(cleveland_data.shape[0]):
    if cleveland_data.iloc[i, 5] > 0:
        cleveland_data.iloc[i, 5] = cleveland_data.iloc[i, 5]
    else:
        cleveland_data.iloc[i, 5] = 0.0

Just like we did with the population, we normalize the number of restaurants using the min/max-method:

In [66]:
cleveland_data['Restaurant_norm'] = ( cleveland_data['Restaurant'] - min(cleveland_data['Restaurant']) ) / ( max(cleveland_data['Restaurant']) - min(cleveland_data['Restaurant']) )

This concludes the "Data Acquisition and Cleaning" Section. <br>
Before we go into the Data Analysis, we will have a last look at our final dataframe:

In [67]:
cleveland_data

,Zip Code,Latitude,Longitude,Population,Population_normalized,Restaurant,Restaurant_norm
0,44102,41.479174,-81.740603,45014,1.000000,12.0,0.461538
1,44103,41.519415,-81.642123,18123,0.324160,9.0,0.346154
2,44104,41.482230,-81.626784,22640,0.437684,3.0,0.115385
3,44105,41.449476,-81.630289,40089,0.876222,6.0,0.230769
4,44106,41.505341,-81.605432,26896,0.544648,23.0,0.884615
5,44109,41.447671,-81.694403,40646,0.890221,8.0,0.307692
6,44110,41.569382,-81.564687,20136,0.374752,4.0,0.153846
7,44113,41.483241,-81.697166,19213,0.351554,22.0,0.846154
8,44108,41.544784,-81.607394,25679,0.514062,3.0,0.115385
9,44111,41.458255,-81.788589,39778,0.868406,14.0,0.538462


### Data Analysis

In this section, we will use the Folium library to visualize different characteristics of the ZIP Codes of Cleveland. <br>
The goal is to distinguish between different types of ZIP Codes in order to make a decision on where to best open a new restaurant.

In [68]:
#install and import the Folium library
!pip install folium
import folium

     |████████████████████████████████| 93 kB 3.4 MB/s  eta 0:00:01


At first, we will have a look at the geographical distribution of the ZIP Codes of Cleveland:

In [69]:
cleveland_centre = [41.4718,-81.6493307]

map_cleveland_zip = folium.Map(location = cleveland_centre, zoom_start = 11)

for zip_, lat, lon in zip(cleveland_data['Zip Code'], cleveland_data['Latitude'], cleveland_data['Longitude']):
    
    label = 'ZIP Code: {}'.format(zip_)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, lon],
                       radius = 8,
                       popup = label,
                       color = 'blue',
                       fill = True,
                       fill_color = '#3186cc',
                       fill_opacity = 0.7,
                       parse_html = False).add_to(map_cleveland_zip)
    
map_cleveland_zip

We see that the ZIP Codes are well-distributed and we do not need to add or leave out some ZIP Codes. <br>
There are ZIP Codes downtown, in the suburban area and also inbetween which means that we have a good basis for our decison-making.

Now we want to add the population to our analysis. We do so by making circles that represent ZIP Codes with high population bigger than circles that represent ZIP Codes with lower population.

In [70]:
cleveland_centre = [41.4718,-81.6493307]

map_cleveland_population = folium.Map(location = cleveland_centre, zoom_start = 11)

for zip_, lat, lon, pop, pop_norm in zip(cleveland_data['Zip Code'], cleveland_data['Latitude'], cleveland_data['Longitude'], cleveland_data['Population'], cleveland_data['Population_normalized']):
    
    label = 'ZIP Code: {} '.format(zip_) + '\n' + 'Population: {}'.format(pop)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, lon],
                       radius = 8 + 30*pop_norm,
                       popup = label,
                       color = 'royalblue',
                       fill = True,
                       fill_color = 'royalblue', #'#3186cc',
                       fill_opacity = 0.5,
                       parse_html = False).add_to(map_cleveland_population)
    
map_cleveland_population

We see that (as usual) the ZIP Codes located downtown have lower population than the ZIP Codes in the suburban area.<br>
However, we also see that there is no direct relation between the population and the distance to the city centre.

In order to get an even more detailed visualization of the different ZIP Codes of Cleveland, we had the amount of restaurants to the visualization. <br>
The amount of restaurants will affect the fill_opacity of the circles, i.e. the higher the amount of restaurants nearby a ZIP Code, the higher the fill_opacity of the circle it is represented by.

In [71]:
cleveland_centre = [41.4718,-81.6493307]

map_cleveland_data = folium.Map(location = cleveland_centre, zoom_start = 11)

for zip_, lat, lon, pop, pop_norm, rest, rest_norm in zip(cleveland_data['Zip Code'], cleveland_data['Latitude'], cleveland_data['Longitude'], cleveland_data['Population'], cleveland_data['Population_normalized'], cleveland_data['Restaurant'], cleveland_data['Restaurant_norm']):
    
    label = 'ZIP Code: {} '.format(zip_) + '\n' + 'Population: {}'.format(pop) + '\n' + 'Restaurants: {}'.format(int(rest))
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, lon],
                       radius = 8 + 30*pop_norm,
                       popup = label,
                       color = 'royalblue',
                       fill = True,
                       fill_color = 'royalblue',
                       fill_opacity = rest_norm + 0.0001,
                       parse_html = False).add_to(map_cleveland_data)
    
for lat, lon, name, cat in zip(cleveland_restaurants['Venue Latitude'], cleveland_restaurants['Venue Longitude'], cleveland_restaurants['Venue'], cleveland_restaurants['Venue Category']):
    
    label = str(name) + '\n' + str(cat)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, lon],
                        radius = 3,
                        popup = label,
                        color = 'cornflowerblue',
                        fill = True,
                        fill_color = 'cornflowerblue',
                        fill_opacity = 1.0,
                        parse_html = False).add_to(map_cleveland_data)
    
map_cleveland_data

Based on this map we can already clearly distinct between different types of ZIP Codes (regarding their population and restaurants).<br>
For example, we see that the ZIP Codes downtown share the characteristic of a high restaurant density with low population. <br>
In order to cleary put the ZIP Codes into different groups, we will execute the KMeans-algorithm on our dataset in the next section.

### Clustering Algorithm

As we do not want to cluster the dataset based on the ZIP Code or the location, we define a new dataframe that only contains the normalized population and amount of restaurants:

In [72]:
cleveland_cluster_data = cleveland_data[['Population_normalized', 'Restaurant_norm']]
cleveland_cluster_data.head()

,Population_normalized,Restaurant_norm
0,1.000000,0.461538
1,0.324160,0.346154
2,0.437684,0.115385
3,0.876222,0.230769
4,0.544648,0.884615


To perform the KMeans-algorithm, we first need to import the according module from Scikit-learn:

In [73]:
#import kmeans module
from sklearn.cluster import KMeans

We apply the KMeans-algorithm on our dataframe defined above (cleveland_cluster_data). <br>
Multiple tries lead to the result that the clustering delivers the "best" result when setting the number of clusters to four. <br>
After the KMeans-algorith has run, we add the Cluster Labels as a new column to the Main Dataframe (cleveland_data).

In [74]:
clusters_n = 4
kmeans = KMeans(n_clusters = clusters_n, random_state=0)
kmeans = kmeans.fit(cleveland_cluster_data)
cleveland_data['Label'] = kmeans.labels_

In [75]:
cleveland_data.head()

,Zip Code,Latitude,Longitude,Population,Population_normalized,Restaurant,Restaurant_norm,Label
0,44102,41.479174,-81.740603,45014,1.000000,12.0,0.461538,3
1,44103,41.519415,-81.642123,18123,0.324160,9.0,0.346154,2
2,44104,41.482230,-81.626784,22640,0.437684,3.0,0.115385,0
3,44105,41.449476,-81.630289,40089,0.876222,6.0,0.230769,3
4,44106,41.505341,-81.605432,26896,0.544648,23.0,0.884615,1


Now we can add the Cluster Labels to our Folium Visualization. Therefore, we define a color for each label. <br>
A circle in the Map will then be visualized in the color that belongs to the cluster of its according ZIP Code.

In [76]:
colors_list = ['darkviolet', 'cyan', 'green', 'red']

In [77]:
cleveland_centre = [41.4718,-81.6493307] #[41.4518,-81.6693307]

map_cleveland_cluster = folium.Map(location = cleveland_centre, zoom_start = 11)

for zip_, lat, lon, pop, pop_norm, rest, rest_norm, cluster in zip(cleveland_data['Zip Code'], cleveland_data['Latitude'], cleveland_data['Longitude'], cleveland_data['Population'], cleveland_data['Population_normalized'], cleveland_data['Restaurant'], cleveland_data['Restaurant_norm'], cleveland_data['Label']):
    
    label = 'ZIP: ' + str(zip_) + ' Cluster {}'.format(cluster) + '\n' + 'Population: {}'.format(pop) + '\n' + 'Restaurants: {}'.format(int(rest)) 
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, lon],
                       radius = 8 + 30*pop_norm,
                       popup = label,
                       color = colors_list[cluster],
                       fill = True,
                       fill_color = colors_list[cluster],
                       fill_opacity = rest_norm + 0.0001,
                       parse_html = False).add_to(map_cleveland_cluster)
    
for lat, lon, name, cat in zip(cleveland_restaurants['Venue Latitude'], cleveland_restaurants['Venue Longitude'], cleveland_restaurants['Venue'], cleveland_restaurants['Venue Category']):
    
    label = str(name) + '\n' + str(cat)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, lon],
                        radius = 3,
                        popup = label,
                        color = 'cornflowerblue',
                        fill = True,
                        fill_color = 'cornflowerblue',
                        fill_opacity = 1.0,
                        parse_html = False).add_to(map_cleveland_cluster)
    
map_cleveland_cluster

### Results

Based on the data analysis and the KMeans-clustering, we can see that the ZIP Codes of Cleveland can be divided into four different groups with the following characteristics:

Cluster 0 (violet): medium high population / low restauraunt density <br>
Cluster 1 (blue): low population / high restaurant density <br>
Cluster 2 (green): medium low population / medium low restaurant density <br>
Cluster 3 (red): high population / medium low restaurant density <br>